<a href="https://colab.research.google.com/github/leaBroe/Deep_Learning_in_Python/blob/main/deep_learning_python_winter_school_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project Overview:

**Abstract:**


### Multi-modal AI Application


### Fulfilling the Criteria


### Gradio App


### Model Card

For your sentiment analysis model, your model card should include:

- **Model Details**: This app incorporates three different models in order to perform sentiment analysis from text images. We first use the python-tesseract OCR tool from the `pytesseract` package to extract text from images. We then perform sentiment analysis and topic classification using `distilbert-base-uncased-finetuned-sst-2-english`, which is available on Hugging Face. Because this model only works with english text, we also use the `m2m100_418M` model from Meta (also available on HuggingFace) to translate the input text when it is not in English.
- **Data**: The distilbert SST2 model is based on the BERT transformer from Google, which was trained on large amounts of english text (from Wikipedia amongst others) in a self-supervised fashion. It was then fine-tuned on the Stanford Sentiment Treebank to enhance performance on sentiment analysis tasks.
- **Performance**: The fine-tuned distilbert model achieves a good score of 91.3 on the GLUE benchmark, but as we will discuss later, might struggle with some specific subtasks/topics. The overall pipeline's performance also depends on the language of the input text. The Many-to-Many translation model used here doesn't perform as well when translating Wolof than when translating German, for example.
- **Ethical Considerations**: Developers highlight the model producing biased predictions which affect underrepresented populations. As the SST 2 dataset was sourced from movie reviews on Rotten Tomatoes, many of the statements on which the fine-tuning is based on contain judgement on the way (American) movies in particular portray one topic or another. Our focus on analyzing sentiment around environmental issues makes it quite likely for geographical information to be of relevance, which may skew results significantly.

### Outlook

Given an extra month, you might consider:

- **Expanding Data Sources**: Including more diverse sources of images and text, such as news articles or blogs, to enrich the analysis.
- **Model Fine-Tuning**: Fine-tuning the sentiment analysis model on a dataset specifically related to environmental discourse to improve accuracy and relevance.
- **Feature Expansion**: Adding functionality to track sentiment trends over time, enabling longitudinal studies on public sentiment toward environmental issues.

In [3]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (9,694 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [4]:
!pip install pytesseract
#pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'

In [5]:
# After installing Tesseract OCR, you need to inform pytesseract of the executable path.
# In Google Colab, it's usually not necessary to manually set the Tesseract path as the apt installation puts
# it in a standard location that pytesseract can automatically find. However, if you do run into issues where
# pytesseract can't find the Tesseract executable, you can set the path manually like this:

import pytesseract
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')


In [6]:
!pip install pytesseract transformers pillow

In [7]:
!pip install opencv-python-headless pillow

In [8]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=576e89027bd341b7b67e5a29f6898c12b6da0e36c4abd56cad41437f373a4588
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [9]:
from google.colab import files
uploaded = files.upload()

# Assuming you now have the file, let's say "image.jpg", uploaded
image_path = next(iter(uploaded))  # This gets the name of the uploaded file


Saving deutsch.png to deutsch (1).png


In [10]:
import cv2
import numpy as np

def binarize_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return binary



In [11]:
from PIL import Image
import pytesseract

# Assuming you have an image named 'example.jpg' in your Colab workspace
image = Image.open(image_path)
binarized_image = binarize_image(image_path)
extracted_text_binarized = pytesseract.image_to_string(binarized_image)
extracted_text = pytesseract.image_to_string(image)
print(extracted_text, extracted_text_binarized)


Dieser Text ist deutsch
 Dieser Text ist deutsch]



In [12]:
import pytesseract
from PIL import Image
from transformers import pipeline
from langdetect import detect

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to extract text from image using pytesseract
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()

# Function to detect the language of the text
def detect_language(text):
    try:
        return detect(text)
    except Exception as e:
        print(f"Error detecting language: {e}")
        return None

# Function to get sentiment from text
def get_sentiment(text):
    results = sentiment_pipeline(text)
    return results

# Example usage
#image_path = "path/to/your/image.jpg"  # Make sure to update this path

# Extract text from the image
extracted_text = extract_text_from_image(image_path)
print(f"Extracted Text: {extracted_text}")

# Detect the language of the extracted text
text_language = detect_language(extracted_text)

# Perform sentiment analysis if the text is in English
if text_language == 'en':
    sentiment_result = get_sentiment(extracted_text)
    print(f"Sentiment Analysis Result: {sentiment_result}")
else:
    print("Sentiment analysis not supported for this language (text has to be in English).")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Extracted Text: Dieser Text ist deutsch
Sentiment analysis not supported for this language (text has to be in English).


In [13]:
import pytesseract
from PIL import Image
from transformers import pipeline
from langdetect import detect

# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Load the emotion detection pipeline
emotion_pipeline = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

# Function to extract text from image using pytesseract
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()


# Function to detect the language of the text
def detect_language(text):
    try:
        return detect(text)
    except Exception as e:
        print(f"Error detecting language: {e}")
        return None

# Function to get sentiment from text
def get_sentiment(text):
    results = sentiment_pipeline(text)
    return results

# Function to get emotion from text
def get_emotion(text):
    results = emotion_pipeline(text)
    return results


# Extract text from the image
extracted_text = extract_text_from_image(image_path)
print(f"Extracted Text: {extracted_text}")

# Detect the language of the extracted text
text_language = detect_language(extracted_text)

# Perform sentiment analysis if the text is in English
if text_language == 'en':
    sentiment_result = get_sentiment(extracted_text)
    print(f"Sentiment Analysis Result: {sentiment_result}")

    # Perform emotion detection on the text
    emotion_result = get_emotion(extracted_text)
    print(f"Emotion Analysis Result: {emotion_result}")
else:
    print("Sentiment and emotion analysis not supported for this language or text is not in English.")


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Extracted Text: Dieser Text ist deutsch
Sentiment and emotion analysis not supported for this language or text is not in English.


In [14]:
!pip install transformers --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [27]:
from transformers import pipeline

# Initialize the translation pipeline
# This example uses a multi-language model. You can replace it with a specific model if preferred.

# old language model, not as good
translation_pipeline = pipeline("translation", model="facebook/m2m100_418M")

# potentially better model
#from transformers import NllbTokenizer
#translation_pipeline = pipeline("translation", model="facebook/nllb-200-distilled-600M")


In [33]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/m2m100_418M"

# Initialize tokenizer and model

#tokenizer = NllbTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer = M2M100Tokenizer.from_pretrained(model_name)

model = M2M100ForConditionalGeneration.from_pretrained(model_name)
#model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Bookmark

In [35]:
import pytesseract
from PIL import Image
from transformers import pipeline
from langdetect import detect

# Load the pipelines
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
emotion_pipeline = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")
translation_pipeline = pipeline("translation", model="facebook/m2m100_418M")
#translation_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-multilingual-en")




# Function to extract text from image using pytesseract
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()

def detect_language(extracted_text):
    detected_lang = detect(extracted_text)
    print(f"Detected language: {detected_lang}")  # For debugging
    return detected_lang

#def translate_text_to_english(extracted_text):
    # The facebook/m2m100_418M model automatically detects the source language
    #translated_text = translation_pipeline(extracted_text, forced_bos_token_id=translation_pipeline.tokenizer.get_lang_id("en"))[0]['translation_text']
    #return translated_text

def translate_text_to_english(text, detected_language):
    # The M2M100 model requires setting the forced_bos_token_id to specify the target language
    tokenizer.src_lang = detected_language  # Example: Assume the source language is French; this should be dynamically determined if possible
    encoded = tokenizer(text, return_tensors="pt")
    # Set the target language ID for English
    #generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["deu_Latn"], max_length=30)
    #translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text


# Function to get sentiment from text
def get_sentiment(extracted_text):
    results = sentiment_pipeline(translated_text)
    return results

# Function to get emotion from text
def get_emotion(extracted_text):
    results = emotion_pipeline(translated_text)
    return results


# Extract and translate text from the image
extracted_text = extract_text_from_image(image_path)
detected_language = detect_language(extracted_text)
translated_text = translate_text_to_english(extracted_text, detected_language)
print(f"Translated Text: {translated_text}")

# Perform sentiment and emotion analysis on the translated text
sentiment_result = get_sentiment(translated_text)
print(f"Sentiment Analysis Result: {sentiment_result}")

emotion_result = get_emotion(translated_text)
print(f"Emotion Analysis Result: {emotion_result}")


Detected language: de
Translated Text: This text is German.
Sentiment Analysis Result: [{'label': 'POSITIVE', 'score': 0.5755502581596375}]
Emotion Analysis Result: [{'label': 'joy', 'score': 0.4265340268611908}]


In [18]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5

In [19]:
from google.colab import files
uploaded = files.upload()

# Assuming you now have the file, let's say "image.jpg", uploaded
image_path = next(iter(uploaded))  # This gets the name of the uploaded file

Saving deutsch.png to deutsch (2).png


In [20]:

def process_image(image):
    extracted_text = extract_text_from_image(image)
    print(f"Extracted Text: {extracted_text}")  # Debug print

    translated_text = translate_text_to_english(extracted_text)
    print(f"Translated Text: {translated_text}")  # Debug print

    sentiment_result = get_sentiment(translated_text)
    print(f"Sentiment: {sentiment_result}")  # Debug print

    emotion_result = get_emotion(translated_text)
    print(f"Emotion: {emotion_result}")  # Debug print

    return extracted_text, translated_text, sentiment_result, emotion_result

process_image(image_path)

Extracted Text: Dieser Text ist deutsch
Translated Text: This text is German
Sentiment: [{'label': 'POSITIVE', 'score': 0.8507675528526306}]
Emotion: [{'label': 'joy', 'score': 0.44141072034835815}]


('Dieser Text ist deutsch',
 'This text is German',
 [{'label': 'POSITIVE', 'score': 0.8507675528526306}],
 [{'label': 'joy', 'score': 0.44141072034835815}])

In [21]:
import gradio as gr

# Your existing function for extracting text from an image file
def extract_text_from_image(image_path):
    from PIL import Image
    import pytesseract
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()

def translate_text_to_english(text):
    # The M2M100 model requires setting the forced_bos_token_id to specify the target language
    tokenizer.src_lang = "fr"  # Example: Assume the source language is French; this should be dynamically determined if possible
    encoded = tokenizer(text, return_tensors="pt")
    # Set the target language ID for English
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

# Function to get sentiment from text
def get_sentiment(text):
    results = sentiment_pipeline(text)
    return results

# Function to get emotion from text
def get_emotion(text):
    results = emotion_pipeline(text)
    return results

def process_image(image_path):
    extracted_text = extract_text_from_image(image_path)
    translated_text = translate_text_to_english(extracted_text)
    sentiment_result = get_sentiment(translated_text)
    emotion_result = get_emotion(translated_text)
    return extracted_text, translated_text, sentiment_result, emotion_result

iface = gr.Interface(fn=process_image,
                     inputs=gr.Image(label="Upload Image", type="filepath"),
                     outputs=[gr.Textbox(label="Extracted Text"),
                              gr.Textbox(label="Translated Text"),
                              gr.Textbox(label="Sentiment Analysis Result"),
                              gr.Textbox(label="Emotion Analysis Result")],
                     title="Image to Sentiment and Emotion Analysis",
                     description="Upload an image containing text, and the app will translate the text to English, then perform sentiment and emotion analysis.")

iface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bbbe3708186b6f9bea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [22]:
!pip install spacy

In [23]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [24]:
import gradio as gr
from PIL import Image
import pytesseract
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Your existing function for extracting text from an image file
def extract_text_from_image(image_path):
    from PIL import Image
    import pytesseract
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()


def translate_text_to_english(text):
    # The M2M100 model requires setting the forced_bos_token_id to specify the target language
    tokenizer.src_lang = "fr"  # Example: Assume the source language is French; this should be dynamically determined if possible
    encoded = tokenizer(text, return_tensors="pt")
    # Set the target language ID for English
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

# Function to get sentiment from text
def get_sentiment(text):
    results = sentiment_pipeline(text)
    return results

# Function to get emotion from text
def get_emotion(text):
    results = emotion_pipeline(text)
    return results

# Function to extract named entities using spaCy
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Main processing function to integrate OCR, translation, sentiment and emotion analysis, and NER
def process_image(image):
    extracted_text = extract_text_from_image(image)
    translated_text = translate_text_to_english(extracted_text)
    sentiment_result = get_sentiment(translated_text)
    emotion_result = get_emotion(translated_text)
    entities = extract_entities(translated_text)  # Use spaCy to extract entities
    entities_str = ', '.join([f"{text} ({label})" for text, label in entities])  # Format entities for display
    return extracted_text, translated_text, sentiment_result, emotion_result, entities_str

# Define Gradio interface
iface = gr.Interface(fn=process_image,
                     inputs=gr.Image(label="Upload Image", type="filepath"),
                     outputs=[gr.Textbox(label="Extracted Text"),
                              gr.Textbox(label="Translated Text"),
                              gr.Textbox(label="Sentiment Analysis Result"),
                              gr.Textbox(label="Emotion Analysis Result"),
                              gr.Textbox(label="Extracted Entities")],
                     title="Image to Sentiment and Emotion Analysis",
                     description="Upload an image containing text, and the app will translate the text to English, then perform sentiment and emotion analysis and extract named entities.")

# Launch the app
# debug=True
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c7cb8cef6285fcb09f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [37]:
import gradio as gr
from PIL import Image
import pytesseract
import spacy

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Your existing function for extracting text from an image file
def extract_text_from_image(image_path):
    from PIL import Image
    import pytesseract
    image = Image.open(image_path)
    extracted_text = pytesseract.image_to_string(image)
    return extracted_text.strip()


from langdetect import detect
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load your translation model and tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

def translate_text_to_english(text):
    # Detect the language of the input text
    detected_lang = detect(text)
    print(f"Detected language: {detected_lang}")  # For debugging

    # Check if the detected language is English
    if detected_lang == 'en':
        return text  # Return the original text if it's already in English

    # Specify the source language for the tokenizer; m2m100 uses language codes
    tokenizer.src_lang = detected_lang

    # Encode the text for the model
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate translation tokens and decode them to text
    # Note: forced_bos_token_id forces the model to translate to English
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_text, detected_lang


# Function to get sentiment from text
def get_sentiment(text):
    results = sentiment_pipeline(text)
    return results

# Function to get emotion from text
def get_emotion(text):
    results = emotion_pipeline(text)
    return results

# Function to extract named entities using spaCy
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Main processing function to integrate OCR, translation, sentiment and emotion analysis, and NER
def process_image(image):
    extracted_text = extract_text_from_image(image)
    translated_text, detected_lang = translate_text_to_english(extracted_text)
    sentiment_result = get_sentiment(translated_text)
    emotion_result = get_emotion(translated_text)
    entities = extract_entities(translated_text)  # Use spaCy to extract entities
    entities_str = ', '.join([f"{text} ({label})" for text, label in entities])  # Format entities for display
    return extracted_text, detected_lang, translated_text, sentiment_result, emotion_result, entities_str

# Define Gradio interface
iface = gr.Interface(fn=process_image,
                     inputs=gr.Image(label="Upload Image", type="filepath"),
                     outputs=[gr.Textbox(label="Extracted Text"),
                              gr.Textbox(label="Detected Language"),
                              gr.Textbox(label="Translated Text"),
                              gr.Textbox(label="Sentiment Analysis Result"),
                              gr.Textbox(label="Emotion Analysis Result"),
                              gr.Textbox(label="Extracted Entities")],
                     title="Image to Sentiment and Emotion Analysis",
                     description="Upload an image containing text, and the app will translate the text to English, then perform sentiment and emotion analysis and extract named entities.")

# Launch the app
# debug=True
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://61ce9f3573e5d902af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
